# Data preparation step

In [11]:
import os
from pathlib import Path

import polars as pl

In [12]:
golden_df = pl.read_csv(Path(os.getcwd()).parent.parent / "data/merged_dataset/golden/train.tsv", separator='\t',
                 quote_char='\"', infer_schema=False)
golden_df

input,output,instruct,dataset_type,dataloader_name
str,str,str,str,str
"""Byte for France або “Мій досві…","""Byte for France або “Мій досві…","""Виправ граматичні помилки в по…","""1""","""UaGecDataset"""
"""Сьогодні розповім про те як і …","""Сьогодні розповім про те<g ed=…","""Виправ граматичні помилки в по…","""1""","""UaGecDataset"""
"""Моє бачення Instagram Колись д…","""Моє бачення Instagram Колись д…","""Виправ граматичні помилки в по…","""1""","""UaGecDataset"""
"""Було це за декілька років до п…","""Було це за <g ed=""кілька"" et=""…","""Виправ граматичні помилки в по…","""1""","""UaGecDataset"""
"""Instagram втілює глибинні бажа…","""Instagram втілює глибинні бажа…","""Виправ граматичні помилки в по…","""1""","""UaGecDataset"""
…,…,…,…,…
"""Заголовок: Леон ГОРЕЦКА: «У ни…","""Основні ключові слова: ['Футбо…","""Із поданого тексту обери голов…","""7""","""NewsKeywordDataset"""
"""Заголовок: Моряка Ейдера могл…","""Основні ключові слова: ['Керче…","""Витягни основні терміни з цьог…","""7""","""NewsKeywordDataset"""
"""Заголовок: ""Укргазвидобування…","""Основні ключові слова: ['Михай…","""Визнач, які слова є найбільш з…","""7""","""NewsKeywordDataset"""


In [13]:
def custom_sample(df, sampling_fractions):
    sampled_dfs = []
    
    for name, fraction in sampling_fractions.items():
        filtered_df = df.filter(pl.col("dataloader_name") == name)
        sampled_df = filtered_df.sample(fraction=fraction, seed=42, shuffle=True) 
        sampled_dfs.append(sampled_df)
    
    return pl.concat(sampled_dfs)

In [29]:
sampling_fractions = {
    "UaGecDataset": 0.95,       # Mostly all (95%)
    "UaSqaudDataset": 0.95,     # Mostly all (95%)
    "NewsTopicClassificationDataset": 0.003,  # Max 3%
    "NewsKeywordDataset": 0.003,  # Max 3%
    'MovaInstPosDataset': 0.65,
    'ZnoDataset': 1.0,
    "WscDataset": 1.0,          # Include all (100%)
    "NerDataset": 1.0           # Include all (100%)
}



data_filtered = golden_df.filter(pl.col("dataloader_name") != "null")


shuffled_df = custom_sample(data_filtered, sampling_fractions).sample(fraction=0.5, shuffle=True)
shuffled_df

input,output,instruct,dataset_type,dataloader_name
str,str,str,str,str
"""Запитання: Який середній дохід…","""51 739 доларів""","""На основі наданого контексту, …","""5""","""UaSqaudDataset"""
"""Запитання: Якими ще іноді вваж…","""зовсім окремими мовами""","""Дай відповідь на запитання, сп…","""5""","""UaSqaudDataset"""
"""Запитання: Який фільм у Парижі…","""Людина-павук 3""","""З урахуванням контексту дай то…","""5""","""UaSqaudDataset"""
"""Фіолетова спина труснулася, ал…","""Фіолетова спина труснулася, ал…","""Виправ граматичні помилки в по…","""1""","""UaGecDataset"""
"""А) виконавиця Б) гардеробщиця …","""Б""","""На основі інформації, наданої …","""8""","""ZnoDataset"""
…,…,…,…,…
"""ПЕРШИЙ З ТРЬОХ ДУХІВ.""","""ПЕРШИЙ <g ed=""ІЗ"" et=""Spelling…","""Виправ граматичні помилки в по…","""1""","""UaGecDataset"""
"""Запитання: Що створює сильний …",null,"""На основі контексту, що надаєт…","""5""","""UaSqaudDataset"""
"""Запитання: З яким італійським …","""Трієстом""","""Давай відповіді на запитання в…","""5""","""UaSqaudDataset"""


In [30]:
shuffled_df['dataloader_name'].value_counts()

dataloader_name,count
str,u32
"""NewsTopicClassificationDataset""",180
"""NerDataset""",139
"""WscDataset""",104
"""UaGecDataset""",2952
"""NewsKeywordDataset""",170
"""UaSqaudDataset""",5335
"""ZnoDataset""",1515
"""MovaInstPosDataset""",1831


In [31]:
artificial_df = pl.read_csv(Path(os.getcwd()).parent.parent / "data/merged_dataset/synthetic/small/train.csv", separator='\t',
                 quote_char='\"', infer_schema=False)
artificial_df

input,output,instruct,dataset_type,dataloader_name
str,str,str,str,str
"""Зараз у назві ізоензимів зазна…","""<p t=""ADV"">Зараз</p> <p t=""ADP…","""Використай наведені нижче інст…","""8""","""PapersDataset"""
"""Головна відмінна риса Peercoin…","""<p t=""ADJ"">Головна</p> <p t=""A…","""Використай наведені нижче інст…","""8""","""PapersDataset"""
"""дає план давнього Острога і за…","""<p t=""VERB"">дає</p> <p t=""NOUN…","""Використай наведені нижче інст…","""8""","""PapersDataset"""
"""МЕДИЧНІ ПЕРСПЕКТИВИ / MEDICNI …","""<p t=""ADJ"">МЕДИЧНІ</p> <p t=""N…","""Використай наведені нижче інст…","""8""","""PapersDataset"""
""", Волженцева І., Калмиков Г. &…","""<p t=""PUNCT"">,</p> <p t=""PROPN…","""Використай наведені нижче інст…","""8""","""PapersDataset"""
…,…,…,…,…
"""BMJ best Practice topics are r…","""<p t=""X"">BMJ</p> <g ed=""Best"" …","""Використай наведені нижче інст…","""8""","""PapersDataset"""
"""classmethod (x) – створює зазн…","""<p t=""X"">classmethod</p> <p t=…","""Використай наведені нижче інст…","""8""","""PapersDataset"""
"""Робота з файлами Перш, ніуж пр…","""<p t=""NOUN"">Робота</p> <p t=""A…","""Використай наведені нижче інст…","""8""","""PapersDataset"""


In [32]:
sampling_fractions = {
    "PapersDataset": 0.6,     
    "UbertextV2Dataset": 1,     
}

data_filtered = artificial_df.filter(pl.col("dataloader_name") != "null")

art_shuffled_df = custom_sample(data_filtered, sampling_fractions).sample(fraction=0.1, shuffle=True)
art_shuffled_df

input,output,instruct,dataset_type,dataloader_name
str,str,str,str,str
"""Окрему увагу привертає робота …","""<p t=""ADJ"">Окрему</p> <p t=""NO…","""Використай наведені нижче інст…","""8""","""PapersDataset"""
"""On the other hand, including p…","""<p t=""X"">On</p> <p t=""X"">the</…","""Використай наведені нижче інст…","""8""","""PapersDataset"""
"""При цій церкві було організова…","""<p t=""ADP"">При</p> <p t=""DET"">…","""Використай наведені нижче інст…","""8""","""UbertextV2Dataset"""
"""Ці «Апос­ толи»— це останній у…","""<p t=""DET"">Ці</p> <p t=""PUNCT""…","""Використай наведені нижче інст…","""8""","""PapersDataset"""
""", ɡɚɬɜɟɪщɠɟɧɢɦɢ ɧɚɤɚɡɨɦ ɆɈɁ ɍɤ…","""<p t=""PUNCT"">,</p> <g ed=""ɡɚɬɜ…","""Використай наведені нижче інст…","""8""","""PapersDataset"""
…,…,…,…,…
""", в якій просить вказані судов…","""<p t=""PUNCT"">,</p> <p t=""ADP"">…","""Використай наведені нижче інст…","""8""","""UbertextV2Dataset"""
"""Підтримка цифросвих моделей ос…","""<p t=""NOUN"">Підтримка</p> <g e…","""Використай наведені нижче інст…","""8""","""PapersDataset"""
"""На думку заявника касаційної с…","""<p t=""ADP"">На</p> <p t=""NOUN"">…","""Використай наведені нижче інст…","""8""","""UbertextV2Dataset"""


In [33]:
art_shuffled_df['dataloader_name'].value_counts()

dataloader_name,count
str,u32
"""PapersDataset""",3637
"""UbertextV2Dataset""",3499


In [34]:
mixed = pl.concat([shuffled_df, art_shuffled_df]).sample(fraction=0.5, shuffle=True)
mixed['dataloader_name'].value_counts()

dataloader_name,count
str,u32
"""ZnoDataset""",778
"""NewsKeywordDataset""",82
"""NerDataset""",63
"""WscDataset""",54
"""PapersDataset""",1816
"""NewsTopicClassificationDataset""",94
"""UaGecDataset""",1436
"""MovaInstPosDataset""",919
"""UaSqaudDataset""",2710


In [38]:
import math

train_df = shuffled_df.head(math.ceil(len(mixed) * 0.85))
valid_df = shuffled_df.tail(math.ceil(len(mixed) * 0.15))

In [40]:
def create_json(row):
    if row["output"] and row["instruct"] and row["input"]:
        return {
            "messages": [
                {"role": "system", "content": "You are a helpful assistant" },
                {"role": "user", "content": f'{row["instruct"]}\n{row["input"]}'},
                {"role": "assistant", "content": row["output"]}
            ]
        }
    return None

In [41]:
from tqdm import tqdm
import json

train_path = Path(Path(os.getcwd()).parent.parent / 'data/merged_dataset/golden/openai/train.jsonl')
valid_path = Path(Path(os.getcwd()).parent.parent / 'data/merged_dataset/golden/openai/valid.jsonl')

with open(train_path, 'w', encoding="utf-8") as file:
    for row in tqdm(train_df.iter_rows(named=True), total=len(train_df), desc="Processing train rows"):
        if json_obj := create_json(row):
            file.write(json.dumps(json_obj, ensure_ascii=False) + '\n')

with open(valid_path, 'w', encoding="utf-8") as file:
    for row in tqdm(valid_df.iter_rows(named=True), total=len(valid_df), desc="Processing valid rows"):
        if json_obj := create_json(row):
            file.write(json.dumps(json_obj, ensure_ascii=False) + '\n')

Processing valid rows: 100%|██████████| 1453/1453 [00:00<00:00, 26125.96it/s]


## OpenAI

#### File loading for pretrain

# TODO: add epoch number

In [42]:
from dotenv import load_dotenv
from pathlib import Path
import os

load_dotenv()

True

In [43]:
openai_folder_root_path = Path(os.getcwd()).parent.parent / 'data/merged_dataset/golden/openai'

In [44]:
train_path = Path(openai_folder_root_path / 'train.jsonl')
valid_path = Path(openai_folder_root_path / 'valid.jsonl')

In [45]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
len(OPENAI_API_KEY)

161

In [46]:
from openai import OpenAI

client = OpenAI()

In [47]:
train_oai_file = client.files.create(
    file=open(train_path, "rb"),
    purpose="fine-tune"
)
valid_oai_file = client.files.create(
    file=open(valid_path, "rb"),
    purpose="fine-tune"
)
train_oai_file, valid_oai_file

(FileObject(id='file-HUB5TQIJag3U5RDtSfZnBZPl', bytes=20144074, created_at=1726225719, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None),
 FileObject(id='file-J1AyMEw1jA7sNenxUmX51YK6', bytes=3686125, created_at=1726225721, filename='valid.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None))

#### Fine tuning

In [48]:
fine_tuning_job = client.fine_tuning.jobs.create(
    training_file=train_oai_file.id,
    validation_file=valid_oai_file.id,
    model='gpt-4o-mini-2024-07-18'
)
fine_tuning_job

FineTuningJob(id='ftjob-LjF5g2Ktf41XxrOnzRvuL9b0', created_at=1726225741, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-Y7mU57w8wKRNhVVhUbD1YvZJ', result_files=[], seed=1475787846, status='validating_files', trained_tokens=None, training_file='file-HUB5TQIJag3U5RDtSfZnBZPl', validation_file='file-J1AyMEw1jA7sNenxUmX51YK6', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [49]:
response = client.fine_tuning.jobs.retrieve(fine_tuning_job.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)


Job ID: ftjob-LjF5g2Ktf41XxrOnzRvuL9b0
Status: validating_files
Trained Tokens: None


In [52]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-LjF5g2Ktf41XxrOnzRvuL9b0
Validating training file: file-HUB5TQIJag3U5RDtSfZnBZPl and validation file: file-J1AyMEw1jA7sNenxUmX51YK6


In [51]:
for f_id in response.result_files:
    result_file = client.files.content(f_id)
    ans = result_file.decode()
    with open(openai_folder_root_path / 'fine_tuning_metrics.csv', 'w') as f:
        f.write(ans)

AttributeError: 'SyncCursorPage[FineTuningJobEvent]' object has no attribute 'result_files'